### Step 1

Open the image and import all the necessary libraries.

Also create a ```show_image``` function for further convenience of image output.

In [1]:
from dataclasses import dataclass
import cv2
from cv2.typing import MatLike as OpenCVImage
import numpy as np

image_path = 'lenna.png'
image = cv2.imread(filename=image_path)

def show_image(image: OpenCVImage, title: str = "Image"):
    cv2.imshow(mat=image, winname=title)


### Step 2

Let's create a dataclass to store multiple values as well as a ```get_dimensions``` function to get the dimensions of an image via ```.shape```

In [251]:
@dataclass
class ImageDimensions:
    width: int
    height: int
    channels: int


def get_dimensions(image: OpenCVImage) -> ImageDimensions:
    height, width, channels = image.shape
    return ImageDimensions(
        width=width, height=height, channels=channels
    )


get_dimensions(image=image)

ImageDimensions(width=512, height=512, channels=3)

### Step 3

Сreate a function ```convert_to_grayscale``` to convert an image to grayscale using ```cv2.COLOR_BGR2GRAY```.

In [ ]:
def convert_to_grayscale(image: OpenCVImage):
    new_image = cv2.cvtColor(
        src=image, code=cv2.COLOR_BGR2GRAY
    )
    return new_image


image = convert_to_grayscale(image=image)
show_image(image=image, title='Grayscale image')

### Step 4

Create a function ```resize_width``` based on which we will recalculate the size via ```cv2.resize``` and ```cv2.INTER_CUBIC```

In [ ]:
def resize_width(image: OpenCVImage, new_width: int):
    height, width = image.shape[:2]  # Trim it down to two values
    proportional = new_width / width  # Calculate the proportion
    new_height = int(height * proportional)
    new_image = cv2.resize(
        src=image,
        dsize=(new_width, new_height),
        interpolation=cv2.INTER_CUBIC,
    )
    return new_image


image = resize_width(
    image=image,
    new_width=200,
)

### Step 5

Crop the face using the ```crop_face``` function based on the selected coordinates of the top left corner and bottom right corner

For convenience of storing coordinates create ```Coordinates``` dataclass.

In [ ]:
@dataclass
class Coordinate:
    x: int
    y: int

def crop_face(image: OpenCVImage, tl: Coordinate, br: Coordinate):
    new_image = image[tl.y:br.y, tl.x:br.x]
    return new_image

top_left_coordinate = Coordinate(x=85, y=80)
bottom_right_coordinate = Coordinate(x=140, y=152)
face_image = crop_face(
    image=image,
    tl=top_left_coordinate,
    br=bottom_right_coordinate,
)
show_image(image=face_image, title='Face image')

### Step 6

Applying median blur to an image using the ```apply_median_blur``` function

In [ ]:
def apply_median_blur(image: OpenCVImage, kernel_size: int):
    new_image = cv2.medianBlur(src=image, ksize=kernel_size)
    return new_image

face_image = apply_median_blur(image=face_image, kernel_size=3)
show_image(image=face_image, title='Median blur face image')

### Step 7

Create a function ```get_gaussian_adaptive_threshold_image``` to use the gaussian adaptive threshold for image via the function ```cv2.adaptiveThreshold```

Create a function ```get_morphological_operations_image``` to use the morphological operations to mask the image via the function ```cv2.morphologyEx```

In [ ]:
def get_gaussian_adaptive_threshold_image(image: OpenCVImage):
    new_image = cv2.adaptiveThreshold(
        src=image,
        maxValue=255,
        adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        thresholdType=cv2.THRESH_BINARY,
        blockSize=11,
        C=2,
    )
    return new_image

def get_morphological_operations_image(image: OpenCVImage, kernel_shape: int = 3):
    kernel = np.ones(shape=(kernel_shape, kernel_shape), dtype=np.uint8)
    new_image = cv2.morphologyEx(
        src=image, op=cv2.MORPH_CLOSE, kernel=kernel,
    )
    return new_image


face_image = get_gaussian_adaptive_threshold_image(image=face_image)
show_image(image=face_image, title='Gaussian adaptive threshold image')
face_image = get_morphological_operations_image(image=face_image)
show_image(image=face_image, title='Morphological operations image')


### Step 8

Perform closing using a larger ```kernel_shape```

In [ ]:
face_image = get_morphological_operations_image(
    image=face_image, kernel_shape=5
)
show_image(image=face_image, title='Closed image')


### Step 9

Сreate a function ```add_salt_and_pepper_noise``` to add salt and pepper noises

In [ ]:
def add_salt_and_pepper_noise(image: OpenCVImage, salt: int = 0.05, pepper: int = 0.05):
    new_image = image.copy()  # Create copy of the image so as not to damage the original image

    # Using randomness, we will generate coordinates of salt and pepper
    coords_salt = [
        np.random.randint(0, i - 1, int(salt * image.size))
        for i in image.shape
    ]
    coords_pepper = [
        np.random.randint(0, i - 1, int(pepper * image.size))
        for i in image.shape
    ]
    new_image[coords_salt[0], coords_salt[1]] = 255
    new_image[coords_pepper[0], coords_pepper[1]] = 0

    return new_image

face_image = add_salt_and_pepper_noise(image=face_image)
show_image(image=face_image, title='Salt and pepper noise image')

### Step 10

Applying median blur to an image in order to remove noise

In [ ]:
face_image = apply_median_blur(image=face_image, kernel_size=3)
show_image(image=face_image, title='Removed noise image')

### Step 11

Create a negative image using the ```get_negative_image``` function

In [ ]:
def get_negative_image(image: OpenCVImage):
    new_image = 255 - image
    return new_image

face_image = get_negative_image(image=face_image)
show_image(image=face_image, title='Negative image')

### Step 12

Create a ```get_edges``` function to get edges via ```cv2.Canny```

In [ ]:
def get_edges(image: OpenCVImage):
    new_image = cv2.Canny(
        image=image, threshold1=50, threshold2=150
    )
    return new_image

edges = get_edges(image=face_image)
show_image(image=edges, title='Edges')

### Step 13

Сreate a function ```save_image``` to save the image

In [ ]:
def save_image(image: OpenCVImage, filename: str):
    cv2.imwrite(filename=filename, img=image)

save_image(image=edges, filename='lenna_edges.png')